# auto_display

This section leverages standard python `ast` (Abstract Syntax Tree) package to implement the `auto_display` function.
These are the main  steps involded:

* Extract the last *statement* in the cell code.
* Decide if it can (or should) be *wrapped* with a call to `display`.
* Perform the wrap.

`auto_display` is the core component of `testcell`. Using `ast` we have robust and consistent parsing even of multi-line python statements.

In [ ]:
#| default_exp core

In [ ]:
#| export
import ast

In [ ]:
#| export
def last_node(code):
    tree = ast.parse(code)
    last_node = None
    for node in ast.walk(tree):
        if isinstance(node, ast.stmt):
            last_node = node
    return last_node

In [ ]:
from fastcore.test import *

In [ ]:
#| export 
def node_source(node,code):
    return ast.get_source_segment(code,node)

In [ ]:
#| test
sample_code = '''
a = 1
b = 2
c = a+b;
# test
'''
test_eq(node_source(last_node(sample_code),sample_code), 'c = a+b')

In [ ]:
#| export
def is_assignment(node):
    return isinstance(node, ast.Assign)

In [ ]:
#| test
test_eq( is_assignment( last_node('a = 1\nb = 2\nc = a+b')), True )
test_eq( is_assignment( last_node('a = 1\nb = 2\nc = a+b\nc')), False )
test_eq( is_assignment( last_node('c')), False )
test_eq( is_assignment( last_node('a=1')),True)
test_eq( is_assignment( last_node('a = function_execution()')),True)
test_eq( is_assignment( last_node('a;')),False)
test_eq( is_assignment( last_node('a')),False)
test_eq( is_assignment( last_node('a - function_execution()')),False)

In [ ]:
#|export
def extract_call(node):
    if not isinstance(node, ast.Expr): return None
    node = node.value # step in
    if isinstance(node, ast.Name): return None # "fn"
    if isinstance(node, ast.Call):
        n = node.func # step in
        if isinstance(n, ast.Name): return n.id # "fn()"
        if isinstance(n, ast.Attribute): return n.attr
    return None # all the rest is not supported

In [ ]:
#| test

# OK
test_eq(extract_call(last_node('fn()')), 'fn')
test_eq(extract_call(last_node('x.fn()')), 'fn')
test_eq(extract_call(last_node('x.y.fn()')), 'fn')
test_eq(extract_call(last_node('x.y.z.fn()')), 'fn')

# KO
test_eq(extract_call(last_node('(fn)')), None)
test_eq(extract_call(last_node('fn')), None)
test_eq(extract_call(last_node('(fn(),fn)')), None)
test_eq(extract_call(last_node('(x.y.fn(),fn())')), None)

In [ ]:
#| export
def is_function_call(node,names):
    function_name = extract_call(node)
    if function_name is None: return False # this is not a function call
    return function_name in names

In [ ]:
#| test
test_eq( is_function_call( last_node('func(123)'), names=['func'] ) , True )
test_eq( is_function_call( last_node('xxx.yyy.zzz.func(123)'), names=['func'] ) , True )
test_eq( is_function_call( last_node('func(123)'), names=['def'] ) , False )
test_eq( is_function_call( last_node('a=func(123)'), names=['func'] ) , False )
test_eq( is_function_call( last_node('a'), names=['func'] ) , False )
test_eq( is_function_call( last_node('x.a'), names=['func'] ) , False )
test_eq( is_function_call( last_node('x.y.z.a'), names=['func'] ) , False )
test_eq( is_function_call( last_node('(a,b())'), names=['func'] ) , False )

In [ ]:
#| export
def is_import_statement(node):
    return isinstance(node, (ast.Import, ast.ImportFrom))

In [ ]:
#| test
test_eq( is_import_statement(last_node('123')) , False )
test_eq( is_import_statement(last_node('func(123)')) , False )
test_eq( is_import_statement(last_node('# test')) , False )
test_eq( is_import_statement(last_node('# import numpy')) , False )
test_eq( is_import_statement(last_node('import numpy')) , True )
test_eq( is_import_statement(last_node('from PIL import Image')) , True )

In [ ]:
#| export
def need_display(node):
    if node is None: return False
    if is_function_call(node,names=['print','display']): return False
    if is_import_statement(node): return False
    return True

In [ ]:
#| test
def test_need_display(code): return need_display(last_node(code))

test_eq( test_need_display('a') , True )
#test_eq( test_need_display('a;') , False ) # This is not supported with ast: we should do it differently
test_eq( test_need_display('func(a)') , True )
test_eq( test_need_display('{1:1,2:2}') , True )
test_eq( test_need_display('display(a)') , False )
test_eq( test_need_display('# xxx') , False )
test_eq( test_need_display('print(a)') , False )
test_eq( test_need_display('import xxx') , False )
test_eq( test_need_display('from xxx import yyy') , False )

In [ ]:
#| export
def wrap_node(node,function_name):
    return ast.Expr(
        value=ast.Call(
            func=ast.Name(id=function_name, ctx=ast.Load()),
            args=[node],
            keywords=[])
        )

In [ ]:
#| test
test_eq( wrap_node(last_node('a'),'display').value.func.id, 'display')

**NOTE**: we need to make the check on `;` semicolon using string because `ast` ignores it.

In [ ]:
#| export
def last_statement_has_semicolon(code):
    t = [x.strip() for x in code.splitlines()]
    t = [x for x in t if not x.startswith('#')]
    return t[-1].endswith(';')

In [ ]:
test_eq( last_statement_has_semicolon('a=1\nb=2') , False )
test_eq( last_statement_has_semicolon('a=1\nb=2;') , True )
test_eq( last_statement_has_semicolon('a=1\nb=2\n# test') , False )
test_eq( last_statement_has_semicolon('a=1\nb=2;\n# test') , True )

In [ ]:
#| export
def code_till_node(code:str,node):
    t = code.splitlines()
    t = t[:node.lineno]
    t[-1] = t[-1][:node.col_offset]
    if len(t[-1])==0: t = t[:-1]
    return '\n'.join(t)

In [ ]:
def do_test_code_till_node(sample_code):
    return code_till_node(sample_code, last_node(sample_code))
    
test_eq( do_test_code_till_node('a=1\na') , 'a=1' ) # two lines
test_eq( do_test_code_till_node('a=1;a') , 'a=1;' ) # inlined
test_eq( do_test_code_till_node('a=1;a\n#') , 'a=1;' ) # with post-comment
test_eq( do_test_code_till_node('a=1;print(\na)') , 'a=1;' ) # multiline instruction
test_eq( do_test_code_till_node('print(1,\n2);print(\na)') , 'print(1,\n2);' ) # all together

`auto_display` is the main function of this module, and it determines whether a given line of code should be wrapped with a `display(...)` statement or not. It returns the modified code with that modification applied if necessary.

In [ ]:
#| export
def auto_display(code):
    if last_statement_has_semicolon(code): return code
    
    n = last_node(code)
    if not need_display(n): return code
    
    ns = node_source(n,code)
    ret = code_till_node(code, last_node(code))
    ret += f'\ndisplay( # %%testcell\n{ns}\n) # %%testcell'
    return ret

In [ ]:
test_eq( auto_display('a=1\na'), 'a=1\ndisplay( # %%testcell\na\n) # %%testcell' )
test_eq( auto_display('a=1\na;'), 'a=1\na;' )

In [ ]:
print(auto_display('a=1\na'))

a=1
display( # %%testcell
a
) # %%testcell


In [ ]:
print(auto_display('a=3\na;'))

a=3
a;


In [ ]:
exec(auto_display('a=5\na'))

5

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()